In [ ]:
import requests
from bs4 import BeautifulSoup
from google.colab import files
import csv
import os
import re
import pandas as pd

In [ ]:
DOMAIN = 'https://www.nfl.com/stats/player-stats/category'

# Lista para armazenar as URLs encontradas
URLs = [f'{DOMAIN}/passing/2024/reg/all/passingyards/desc',
        f'{DOMAIN}/passing/2024/REG/all/passingyards/DESC?aftercursor=AAAAGQAAABlAoMIAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNVFExSWl3aU16SXdNRFF6TkRFdE5USXpOUzA0TnprM0xUTmlPV0V0TUdReU56RXhZekJpWXpJM0lpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/passing/2024/REG/all/passingyards/DESC?aftercursor=AAAAMgAAADJAdpAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXpOakVpTENJek1qQXdORFEwWmkwME1qTXdMVEl6TmpBdE9HVXhOQzFrWW1KbE1XUmxaR1V4TWpBaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/passing/2024/REG/all/passingyards/DESC?aftercursor=AAAASwAAAEtANgAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNaUlzSWpNeU1EQTBZVFExTFRRMk1qWXRPVEk0Tnkxak5EVTBMVFEzT1dJNU5ESmpZbVF5TmlJc0lqSXdNalFpWFgwPQ==',
        f'{DOMAIN}/passing/2024/REG/all/passingyards/DESC?aftercursor=AAAAZAAAAFsAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdJaXdpTXpJd01EUmpOREV0TldFd05DMHpOekUwTFdObVlqY3RPRGN5WWpJd1lqQmpaVEV6SWl3aU1qQXlOQ0pkZlE9PQ=='
]

HEADERS = {'Uer-Agent': 'Mozilla/5.0'}

In [ ]:
def get_page_content(url, headers):
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')
  return soup

In [ ]:
# Lista para armazenar as URLs dos jogadores e das imagens corrigidas
url_players = []

# Loop sobre todas as URLs principais
for url in URLs:
    # Obter o conteúdo da página para a URL atual
    soup = get_page_content(url, HEADERS)

    # Localizar a tabela de jogadores na página
    table = soup.find('table', {'class': 'd3-o-table d3-o-table--detailed d3-o-player-stats--detailed d3-o-table--sortable'})

    # Iterar sobre todas as linhas que contêm os links para os jogadores
    for row in table.find_all('a', {'class': 'd3-o-player-fullname nfl-o-cta--link'}):
        # Adicionar o link de cada jogador à lista
        url_players.append(row['href'])

In [ ]:
# Função para escrever dados no arquivo CSV
def write_to_csv(filename, data, headers):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for row in data:
            writer.writerow(row)

# Listas para armazenar os dados dos QBs
qb_data = []

# Loop sobre todas as URLs dos jogadores
for url in url_players:
    # Coletar informações do time do jogador em 2024
    soup = get_page_content(f'https://www.nfl.com{url}stats/career', HEADERS)

    # Extrair a posição do jogador
    position_tag = soup.find('span', {'class': 'nfl-c-player-header__position'})
    player_position = position_tag.text.strip() if position_tag else 'Unknown'

    # Extrair o time do jogador
    team_tag = soup.find_all('td')
    player_team = team_tag[1].text.strip() if team_tag else 'Unknown'

    # Dados do time em 2024
    soup = get_page_content(f'https://www.nfl.com{url}stats', HEADERS)

    table = soup.find('table')
    rows = table.find_all('tr')

    if player_position == "QB":
        for row in rows:
            # Extrair as células da linha
            cells = row.find_all('td')
            if len(cells) > 0:
                game_data = [
                    player_team,           # Nome do time
                    cells[0].text.strip(), # WK
                    cells[3].text.strip(), # COMP
                    cells[4].text.strip(), # ATT
                    cells[5].text.strip(), # YDS
                    cells[11].text.strip(), # RATE
                ]
                qb_data.append(game_data)

# Salvar os dados em arquivos CSV
write_to_csv('Yards.csv', qb_data, [
    'team', 'week', 'completions', 'attempts', 'yards', 'rate'
])